# Email Service - Send to all customers

### Download dependencies

In [1]:
%pip install openai bs4 pandas requests

### Load Zero Shot prompt

In [1]:
with open("/prompts/political_text_classifer", "r", encoding="utf-8") as file:
    instructions = file.read()

FileNotFoundError: [Errno 2] No such file or directory: '/prompts/political_text_classifer'

### Define function to run the political text classifer

In [ ]:
from openai import OpenAI
client = OpenAI()

def classifyText(input_text: str):
    response = client.chat.completions.create(
        model="gpt-4o",
        instructions=instructions,
        input=input_text
    )

    return response

### Run webscraper to get NYC legislation
Beautifulsoup will be used to access HTML of NYC Legistar website. We will save the scrapped data in a Pandas dataframe for further processing

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

request_url = "https://legistar.council.nyc.gov/Calendar.aspx?Mode=This+Week"
web_html = requests.get(request_url).text
soup = BeautifulSoup(web_html, "html.parser")
table = soup.find('table', id='ctl00_ContentPlaceHolder1_gridCalendar_ctl00')

# --- Parse data rows ---
rows = []
for tr in table.find_all('tr')[1:]:  
    cells = tr.find_all('td')
    date = cells[0].get_text(strip=True)
    committee = cells[1].get_text(strip=True)
        
    # Get the agenda link (third column - find <a> within <span>)
    span = cells[2].find('span')
    agenda_link = span.find('a') if span else None
    agenda_url = agenda_link['href'] if agenda_link and agenda_link.has_attr('href') else ''
        
    # Make sure the URL is absolute
    if agenda_url and not agenda_url.startswith('http'):
        agenda_url = 'https://legistar.council.nyc.gov/' + agenda_url.lstrip('/')
        
    rows.append({
            'Date': date,
            'Committee': committee,
            'Agenda Link': agenda_url
    })

# --- Build DataFrame ---
df = pd.DataFrame(rows)
print(df)

                                                 Date   Committee  \
0                         City Council Stated Meeting  11/25/2025   
1                Committee on Women and Gender Equity  11/25/2025   
2                        Committee on General Welfare  11/25/2025   
3                                Committee on Finance  11/25/2025   
4                             Committee on Technology  11/25/2025   
5   Committee on Mental Health, Disabilities and A...  11/25/2025   
6                               Committee on Land Use  11/25/2025   
7   Committee on Governmental Operations, State & ...  11/25/2025   
8                 Committee on Civil and Human Rights  11/25/2025   
9   Subcommittee on Landmarks, Public Sitings and ...  11/25/2025   
10              Subcommittee on Zoning and Franchises  11/25/2025   
11                                 Committee on Aging  11/25/2025   
12       Committee on Rules, Privileges and Elections  11/25/2025   
13               Committee on Civi